<a href="https://colab.research.google.com/github/tusharsingh-spring/machine-learning-new/blob/main/CHAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "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"
with open(".env", "w") as f:
    f.write("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")
!pip install -q --upgrade langchain langchain-community


In [2]:
!pip install python-dotenv
from google.colab import userdata
userdata.get('chatkey')

'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'

In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access variables
chat_key= os.getenv("OPENAI_API_KEY")



print(chat_key[4:] + "....")  # print only first 4 chars for security


r-v1-a26063f26cb140b20670474da2f048e368cc26c4ab5a876b2f4fa6eb0a4cabf4....


In [12]:
# Install OpenAI Python package if not already installed
!pip install --upgrade openai
import time
time.sleep(60)
# Import required library
from openai import OpenAI
import getpass
import os
OPENROUTER_API_KEY= getpass.getpass("Enter your OpenRouter API key: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")
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)

completion = client.chat.completions.create(

  extra_body={},
  model="openai/gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)
print(completion.choices[0].message.content)

Enter your OpenRouter API key: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··········
The meaning of life is a profound and often personal question that has been explored by philosophers, theologians, scientists, and thinkers throughout history. Different cultures, belief systems, and individuals offer various interpretations:

1. **Philosophical Perspectives**: Many philosophical traditions ponder the nature of existence and purpose. Existentialists, for instance, suggest that life has no inherent meaning, and it is up to each individual to create their own purpose.

2. **Religious Interpretations**: Many religions provide a framework for understanding life's purpose. For example, in Christianity, believers may find meaning through a relationship with God and following the teachings of Jesus. In Buddhism, the focus might be on achieving enlightenment and understanding the nature of suffering.

3. **Scientific Views**: From a scientific viewpoint, some argue

In [13]:
import os
from langchain.document_loaders import CSVLoader
def read_doc(directory):
  csv_loader = CSVLoader(directory, encoding="utf-8")
  documents= csv_loader.load()
  return documents

In [14]:
import pandas as pd
from google.colab import files

# Upload CSV file
uploaded = files.upload()


Saving profiles_summary.csv to profiles_summary.csv


In [16]:
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load your CSV
df = pd.read_csv("profiles_summary.csv")

# Count how many times each platform_number occurs
counts = df['platform_number'].value_counts()

# Keep only rows where platform_number occurs 5 times or less
df_filtered = df[df['platform_number'].isin(counts[counts <= 70].index)]

# Optionally reset index
df_filtered = df_filtered.reset_index(drop=True)

# Save filtered CSV (optional)
df_filtered.to_csv("argo_filtered.csv", index=False)

print(f"Original rows: {len(df)}, Filtered rows: {len(df_filtered)}")


Original rows: 345, Filtered rows: 345


In [18]:
df.columns


Index(['platform_number', 'cycle_number', 'time', 'latitude', 'longitude',
       'temp_mean', 'temp_min', 'temp_max', 'psal_mean', 'psal_min',
       'psal_max', 'pres_max', 'nitrate_mean'],
      dtype='object')

In [19]:
from tqdm import tqdm

# Define summary function (keep narrative text, not metadata)
def summarize_row(row):
    return f"""
On float platform {row['platform_number']} recorded a profile during cycle {row['cycle_number']}
The float was located at latitude {row['latitude']} and longitude {row['longitude']} at time {row['time']}.
Sensor readings included:
- Temperature: mean {row['temp_mean']}°C ,max {row['temp_max']}°C ,min {row['temp_min']}°C
- Salinity:  mean {row['psal_mean']}PSU ,max {row['psal_max']}PSU ,min {row['psal_min']}PSU
- Pressure:  max  {row['psal_max']}psl
- Nitrate:   mean {row['nitrate_mean']} µmol/L
""".strip()

# Apply to DataFrame
tqdm.pandas()
df_filtered["summary"] = df_filtered.progress_apply(summarize_row, axis=1)

100%|██████████| 345/345 [00:00<00:00, 19827.83it/s]


In [20]:
print(df_filtered["summary"].head())

0    On float platform 1901514 recorded a profile d...
1    On float platform 1901759 recorded a profile d...
2    On float platform 1901760 recorded a profile d...
3    On float platform 1901761 recorded a profile d...
4    On float platform 1901762 recorded a profile d...
Name: summary, dtype: object


In [21]:
def chunk_data(summaries, chunk_size=800, chunk_overlap=70):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(summaries)
  return docs

In [22]:
summaries = df_filtered["summary"]
docs = [Document(page_content=s, metadata={}) for s in summaries]

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Split the documents into smaller chunks
chunked_docs = text_splitter.split_documents(docs)

print(f"Number of chunks: {len(chunked_docs)}")
print("Example chunk:\n", chunked_docs[0].page_content)

Number of chunks: 345
Example chunk:
 On float platform 1901514 recorded a profile during cycle 484.0
The float was located at latitude 1.623 and longitude 44.638 at time 2025-01-08 01:24:59.
Sensor readings included:
- Temperature: mean 25.941°C ,max 25.941°C ,min 25.941°C 
- Salinity:  mean 0.013PSU ,max 0.013PSU ,min 0.013PSU
- Pressure:  max  0.013psl
- Nitrate:   mean 27.00142394026461 µmol/L


In [26]:
from langchain.embeddings import OpenAIEmbeddings
embedding = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])

In [27]:
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7936f3924290>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7936f381e8d0>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='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', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

In [29]:
!pip install --upgrade openai

from openai import OpenAI
import getpass

OPENROUTER_KEY = getpass.getpass("Enter your OpenRouter API key: ")

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_KEY
)

# Example chat completion
completion = client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello, how are you?"}]
)

print(completion.choices[0].message.content)


Enter your OpenRouter API key: ··········
Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [37]:
!pip install -qU langchain-openai
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [57]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS



In [62]:
def get_vectorstore(chunked_docs):
    embeddings = OpenAIEmbeddings()
    texts = [str(doc) for doc in chunked_docs]
    vectorstore = FAISS.from_texts(text= chunked_docs, embeddings=embeddings)



In [74]:
# Install FAISS CPU version (most compatible)
!pip install faiss-cpu

# Or if you have GPU support:
# !pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 43.6 MB/s eta 0:00:00


In [75]:
# First install FAISS
!pip install faiss-cpu langchain_community sentence-transformers

# Now import and create the vectorstore
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

def get_vectorstore(chunked_docs):
    """
    Create a FAISS vectorstore from chunked documents with metadata support
    """
    texts = []
    metadatas = []

    # Process each document chunk
    for i, doc in enumerate(chunked_docs):
        # Extract text content
        if hasattr(doc, 'page_content') and hasattr(doc, 'metadata'):
            # LangChain Document object
            texts.append(doc.page_content)
            # Preserve existing metadata and add some tracking info
            metadata = doc.metadata.copy() if hasattr(doc, 'metadata') and doc.metadata else {}
            metadata.update({
                'chunk_id': f"chunk_{i}",
                'processed': True,
                'embedding_model': 'all-MiniLM-L6-v2'
            })
            metadatas.append(metadata)

        elif isinstance(doc, dict):
            # Dictionary format
            if 'text' in doc:
                texts.append(doc['text'])
                metadata = doc.get('metadata', {}).copy()
            else:
                texts.append(str(doc))
                metadata = {}
            metadata.update({
                'chunk_id': f"chunk_{i}",
                'processed': True
            })
            metadatas.append(metadata)

        else:
            # String or other object
            texts.append(str(doc))
            metadatas.append({
                'chunk_id': f"chunk_{i}",
                'content_type': 'summary_chunk',
                'processed': True,
                'embedding_model': 'all-MiniLM-L6-v2'
            })

    # Use local HuggingFace embeddings (no API required)
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'},  # Use CPU for compatibility
        encode_kwargs={'normalize_embeddings': True}
    )

    # Create the FAISS vectorstore
    vectorstore = FAISS.from_texts(
        texts=texts,
        embedding=embeddings,
        metadatas=metadatas
    )

    return vectorstore

# Now use the function
try:
    vectorstore = get_vectorstore(chunked_docs)
    print("✅ Vectorstore created successfully!")
    print(f"📊 Number of documents: {vectorstore.index.ntotal}")

    # Save it
    vectorstore.save_local("summary_vectorstore")
    print("💾 Vectorstore saved as 'summary_vectorstore'")

    # Test with a sample query
    if vectorstore.index.ntotal > 0:
        results = vectorstore.similarity_search("summary", k=min(3, vectorstore.index.ntotal))
        print("\n🔍 Sample search results:")
        for i, result in enumerate(results):
            print(f"{i+1}. {result.page_content[:100]}...")
            print(f"   Metadata: {result.metadata}")
            print()

except Exception as e:
    print(f"❌ Error: {e}")
    print("Make sure FAISS is installed: !pip install faiss-cpu")

✅ Vectorstore created successfully!
📊 Number of documents: 345
💾 Vectorstore saved as 'summary_vectorstore'

🔍 Sample search results:
1. On float platform 2902766 recorded a profile during cycle 185.0
The float was located at latitude 15...
   Metadata: {'chunk_id': 'chunk_95', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

2. On float platform 6903141 recorded a profile during cycle 287.0
The float was located at latitude 23...
   Metadata: {'chunk_id': 'chunk_281', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

3. On float platform 5906634 recorded a profile during cycle 139.0
The float was located at latitude -2...
   Metadata: {'chunk_id': 'chunk_244', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}



In [76]:
# After creating your vectorstore, use these methods to inspect it
print("🔍 Inspecting Vector Database:")

# Check basic info
print(f"Total vectors stored: {vectorstore.index.ntotal}")
print(f"Vector dimension: {vectorstore.index.d}")

# Get all documents and metadata
all_docs = vectorstore.docstore._dict
print(f"Number of documents: {len(all_docs)}")

# Display first few documents with metadata
print("\n📄 First 5 documents with metadata:")
for i, (doc_id, doc) in enumerate(list(all_docs.items())[:5]):
    print(f"Document {i+1} (ID: {doc_id}):")
    print(f"  Content: {doc.page_content[:100]}...")
    print(f"  Metadata: {doc.metadata}")
    print()

🔍 Inspecting Vector Database:
Total vectors stored: 345
Vector dimension: 384
Number of documents: 345

📄 First 5 documents with metadata:
Document 1 (ID: 361552ea-13f7-499b-a7b5-abfbcc56ccca):
  Content: On float platform 1901514 recorded a profile during cycle 484.0
The float was located at latitude 1....
  Metadata: {'chunk_id': 'chunk_0', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

Document 2 (ID: edc3438c-c9f6-47db-ba47-fb42d09429c3):
  Content: On float platform 1901759 recorded a profile during cycle 139.0
The float was located at latitude -2...
  Metadata: {'chunk_id': 'chunk_1', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

Document 3 (ID: ed251b5b-42f4-498d-ae8e-ef73b90eca81):
  Content: On float platform 1901760 recorded a profile during cycle 139.0
The float was located at latitude -2...
  Metadata: {'chunk_id': 'chunk_2', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

Document 4 (ID: 8ac4208f-5304-4aef-b07b-239127fd2cdb):
  Content: 

In [77]:

!pip install langchain openai transformers torch

In [78]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
import os

# Set your OpenRouter API key
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

In [79]:
def load_vectorstore(vectorstore_path="summary_vectorstore"):
    """Load your saved vectorstore"""
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    vectorstore = FAISS.load_local(
        vectorstore_path,
        embeddings,
        allow_dangerous_deserialization=True
    )
    return vectorstore

# Load your vectorstore
vectorstore = load_vectorstore()
print("✅ Vectorstore loaded successfully!")

✅ Vectorstore loaded successfully!


In [80]:
from langchain_community.chat_models import ChatOpenAI

def get_llm():
    """Initialize LLM with OpenRouter"""
    llm = ChatOpenAI(
        model="openai/gpt-4o-mini",  # or any model supported by OpenRouter
        openai_api_base="https://openrouter.ai/api/v1",
        openai_api_key="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",
        temperature=0.1,
        max_tokens=1000
    )
    return llm

In [81]:
def create_rag_system(vectorstore, llm):
    """Create a complete RAG system"""

    # Custom prompt template for better responses
    prompt_template = """you are an Ocean researcher,Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: {context}

    Question: {question}

    Answer:"""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # Create retriever
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 4}  # Retrieve 4 most relevant chunks
    )

    # Create RAG chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )

    return qa_chain

# Initialize LLM (choose one)
llm = get_llm()  # Using OpenRouter
# llm = get_local_llm()  # Using local model

# Create RAG system
rag_system = create_rag_system(vectorstore, llm)
print("✅ RAG system created successfully!")

✅ RAG system created successfully!


/tmp/ipython-input-2790383378.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [82]:
def rag_chat(rag_system, question):
    """Ask a question to the RAG system"""
    print(f"Question: {question}")
    print(" Thinking...")

    try:
        result = rag_system({"query": question})

        print(f"Answer: {result['result']}")
        print("\n Sources used:")

        for i, source_doc in enumerate(result['source_documents']):
            print(f"{i+1}. {source_doc.page_content[:150]}...")
            print(f"   Metadata: {source_doc.metadata}")
            print()

    except Exception as e:
        print(f"Error: {e}")

# Test the RAG system
test_questions = [
    "What are the main summaries in this document?",
    "Can you give me a brief overview of the content?",
    "What are the key points discussed?",
    "Summarize the main topics covered."
]

for question in test_questions:
    rag_chat(rag_system, question)
    print("=" * 80)

🧠 Question: What are the main summaries in this document?
🤔 Thinking...


/tmp/ipython-input-4020677249.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_system({"query": question})


✅ Answer: The document provides profiles recorded by four different float platforms, detailing their locations, sensor readings, and the time of recording. Here are the main summaries:

1. **Float Platform 1901760**:
   - Location: Latitude -26.7285, Longitude 57.647
   - Time: 2025-01-06 21:19:39
   - Temperature: Mean 9.94°C, Max 28.52°C, Min 2.44°C
   - Salinity: Mean 34.87 PSU, Max 35.69 PSU, Min 34.43 PSU
   - Nitrate: Mean 25.40 µmol/L

2. **Float Platform 6903141**:
   - Location: Latitude 23.0586, Longitude 59.2659
   - Time: 2025-01-06 09:48:30
   - Temperature: Mean 14.73°C, Max 25.03°C, Min 6.62°C
   - Salinity: Mean 35.91 PSU, Max 36.67 PSU, Min 35.17 PSU
   - Nitrate: Mean 25.37 µmol/L

3. **Float Platform 1902479**:
   - Location: Latitude -2.2331, Longitude 78.5921
   - Time: 2025-01-02 16:59:17
   - Temperature: Mean 8.13°C, Max 28.02°C, Min 2.72°C
   - Salinity: Mean 34.91 PSU, Max 35.27 PSU, Min 34.78 PSU
   - Nitrate: Mean 25.53 µmol/L

4. **Float Platform 6903139**:

In [85]:
test_questions = [
    "whats the location of platform 1901514 "

]

for question in test_questions:
    rag_chat(rag_system, question)
    print("=" * 80)

🧠 Question: whats the location of platform 1901514 
🤔 Thinking...
✅ Answer: The location of platform 1901514 is at latitude 1.623 and longitude 44.638.

📚 Sources used:
1. On float platform 1901514 recorded a profile during cycle 484.0
The float was located at latitude 1.623 and longitude 44.638 at time 2025-01-08 01:24:...
   Metadata: {'chunk_id': 'chunk_0', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

2. On float platform 1901919 recorded a profile during cycle 95.0
The float was located at latitude -9.2339 and longitude 40.9451 at time 2025-01-09 01:2...
   Metadata: {'chunk_id': 'chunk_17', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

3. On float platform 1901760 recorded a profile during cycle 139.0
The float was located at latitude -26.7285 and longitude 57.647 at time 2025-01-06 21:...
   Metadata: {'chunk_id': 'chunk_2', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

4. On float platform 1901839 recorded a profile during cycle 301.0
The 

In [86]:
test_questions = [
    "whats the salinity profiles near the equator"
]

for question in test_questions:
    rag_chat(rag_system, question)
    print("=" * 80)

🧠 Question: whats the salinity profiles near the equator
🤔 Thinking...
✅ Answer: The salinity profiles near the equator, based on the provided data, are as follows:

1. Float platform 6903031 (located at latitude -5.6815) recorded a mean salinity of 34.8558 PSU, with a maximum of 35.165 PSU and a minimum of 34.619 PSU.

2. Float platform 1902477 (located at latitude -1.4773) recorded a mean salinity of 34.9423 PSU, with a maximum of 35.241 PSU and a minimum of 34.771 PSU.

These profiles indicate that the salinity values near the equator are relatively similar, ranging from approximately 34.85 to 34.94 PSU.

📚 Sources used:
1. On float platform 6903140 recorded a profile during cycle 288.0
The float was located at latitude 23.95372833333333 and longitude 62.675043333333335 a...
   Metadata: {'chunk_id': 'chunk_278', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

2. On float platform 4903660 recorded a profile during cycle 81.0
The float was located at latitude 15.2476445000

In [87]:
test_questions = [
    "Compare BGC parameter in the arabian sea"

]

for question in test_questions:
    rag_chat(rag_system, question)
    print("=" * 80)

🧠 Question: Compare BGC parameter in the arabian sea
🤔 Thinking...
✅ Answer: I don't know.

📚 Sources used:
1. On float platform 3902339 recorded a profile during cycle 6.0
The float was located at latitude -26.02022 and longitude 70.02385 at time 2025-01-06 01...
   Metadata: {'chunk_id': 'chunk_148', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

2. On float platform 2903893 recorded a profile during cycle 37.0
The float was located at latitude 1.5 and longitude 79.75 at time 2025-01-08 14:18:33.
...
   Metadata: {'chunk_id': 'chunk_137', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

3. On float platform 2903876 recorded a profile during cycle 6.0
The float was located at latitude -23.023288333333333 and longitude 49.79074166666667 at...
   Metadata: {'chunk_id': 'chunk_134', 'processed': True, 'embedding_model': 'all-MiniLM-L6-v2'}

4. On float platform 2902185 recorded a profile during cycle 337.0
The float was located at latitude -20.866 and longitude 60.7

In [91]:
test_questions = [
    "compare BGC prameter between agro floats"

]

for question in test_questions:
    rag_chat(rag_system, question)
    print("=" * 80)

🧠 Question: compare BGC prameter between agro floats
🤔 Thinking...
❌ Error: Error code: 401 - {'error': {'message': 'User not found.', 'code': 401}}
